In [ ]:
import os
import sys
path2add = os.path.dirname(os.getcwd())
sys.path.append(path2add)

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader
from langchain_chroma import Chroma
from langchain.embeddings import OllamaEmbeddings

from typing import List

语音模块

In [ ]:
from AIDCM.frontend.audio_gradio.ali_stt_voice_awake import lingji_stt_gradio_va
from AIDCM.frontend.audio_gradio.zijie_tts import tts

def lingji_stt_gradio_va2(inputs):
    return lingji_stt_gradio_va()


检索小模型

In [ ]:
def audio_produce(documents:List):
    '''把document的page_content转换成语音，然后再把路径存储在metadata中。'''
    for doc in documents:
        audio_path = tts(doc.page_content)
        doc.metadata['audio_path'] = audio_path

In [ ]:
text_loader = TextLoader("responses.txt")
data = text_loader.load()

text_splitter = CharacterTextSplitter(separator='\n\n', chunk_size=0, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
audio_produce(all_splits)
local_embeddings = OllamaEmbeddings(base_url="http://172.17.0.2:11434", model="nomic-embed-text")
vector_store = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

retriever = vector_store.as_retriever(search_kwargs={'k': 1})

In [22]:
chain = lingji_stt_gradio_va2 | retriever

In [ ]:
# chain.invoke(input=None)

语音播报

In [25]:
import time
from pygame import mixer

In [15]:
def get_audio_path(documents:List):
    return documents[0].metadata['audio_path']

In [16]:
def play_audio(audio_path):
    mixer.init()
    mixer.music.load(audio_path)
    mixer.music.play()
    while mixer.music.get_busy():
        time.sleep(0.001)
    
    mixer.music.unload()
    mixer.quit()    

最终成品

In [23]:
chain = chain | get_audio_path | play_audio

In [ ]:
chain.invoke(None)

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

transparent_node = RunnablePassthrough()
runnable = RunnableParallel(
    passed=transparent_node,
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [5]:
transparent_node.dict()

{'name': None, 'input_type': None, 'func': None, 'afunc': None}